In [3]:
import sys
sys.path.append('../resources/')

from get_data import stations

In [4]:
import pandas as pd
import numpy as np

In [5]:
interested_stations = ['13180000', '13300000', '13405000', '13410000', '13450000', '13470000', '13550000', '13600002', 
'13650000', '13710001', '13740000', '13750000', '13870000', '13880000', '13885000', '13886000']

df, nodata = stations(interested_stations, '3')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:37<00:00,  2.36s/it]


In [6]:
df.iloc[25000:29000, :]

,13180000,13300000,13405000,13410000,13450000,13470000,13550000,13600002,13650000,13710001,13740000,13750000,13870000,13880000,13885000,13886000
2000-04-12,866.602,330.039,451.538,3291.76,32.3064,59.996,180.448,453.800,584.969,3711.73,1750.21,7129.42,10670.00,11349.2,NaN,NaN
2000-04-13,894.901,313.808,389.640,2992.26,30.8023,52.352,165.801,426.761,563.222,3620.99,1744.18,7270.09,10620.20,11337.3,NaN,NaN
2000-04-14,931.029,333.557,320.521,2824.80,29.8189,57.812,153.378,405.807,543.941,3503.40,1740.16,7287.78,10537.60,11331.4,NaN,NaN
2000-04-15,983.028,354.966,258.935,2735.30,34.8908,77.048,146.447,394.940,527.463,3365.65,1740.16,7305.49,10471.70,11384.9,NaN,NaN
2000-04-16,1076.070,361.003,205.076,2632.55,40.9162,92.182,149.032,373.546,511.355,3208.92,1744.18,7323.23,10406.00,11390.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-03-21,NaN,353.284,NaN,4112.01,156.3440,220.706,492.438,701.754,1332.420,5282.98,1484.78,7599.07,4593.30,10874.8,543.301,2046.46
2011-03-22,NaN,565.960,666.921,3935.54,147.0400,186.103,468.209,663.355,1398.380,5484.28,1496.38,7688.33,4640.58,10909.5,549.295,2062.36
2011-03-23,NaN,813.927,683.705,3883.29,118.3750,111.207,353.042,713.875,1447.710,5626.50,1503.16,7730.82,4659.57,10944.3,556.826,2078.33
2011-03-24,NaN,997.006,676.662,3941.37,111.3300,141.273,273.581,652.557,1499.130,5711.16,1507.03,7745.02,4704.07,10979.2,564.399,2087.49


In [7]:
def get_time_series_for_window(start_date, end_date, station, dataframe):
    return dataframe.loc[start_date:end_date, [station]]

In [8]:
get_time_series_for_window('2011-03-21', '2011-03-29', '13450000', df)

,13450000
2011-03-21,156.344
2011-03-22,147.040
2011-03-23,118.375
2011-03-24,111.330
2011-03-25,115.704
2011-03-26,119.723
2011-03-27,135.628
2011-03-28,191.532
2011-03-29,370.045


In [9]:
def check_nan_values(dataframe):
    return dataframe.isnull().values.any()


In [10]:
check_nan_values(df)

True

In [11]:
check_nan_values(get_time_series_for_window('2011-03-21', '2011-03-29', '13450000', df))

False

In [12]:
check_nan_values(get_time_series_for_window('2000-01-01', '2015-12-31', '13450000', df))

True

In [69]:
def get_valid_sequences(df):
    valid_sequences = []
    starting_idx = 0

    for i, (_, row) in enumerate(df.iterrows()):
        flow = row[0]

        if np.isnan(flow):

            if starting_idx < i-1:
                valid_sequences.append((starting_idx, i))
                starting_idx = i+1
            else:
                starting_idx = i+1
                continue
    
    return valid_sequences

In [82]:
def valid_seqs_minimum_len(valid_seqs, seq_len):
    
    valid_seqs_min_len = []
    pops = []
    for i, (start, end) in enumerate(valid_seqs):
        if end - start >= seq_len:
            valid_seqs_min_len.append((start, end))

    return valid_seqs_min_len

In [83]:
df1 = get_time_series_for_window('2000-01-01', '2015-12-31', '13450000', df)
valid_sequences = get_valid_sequences(df1)

In [87]:
val_seq = valid_seqs_minimum_len(valid_sequences, 10)

In [93]:
start, end = val_seq[3]
df1.iloc[start:end, :]

,13450000
2009-04-04,27.9669
2009-04-05,30.4028
2009-04-06,100.8100
2009-04-07,54.3006
2009-04-08,147.2460
2009-04-09,101.7240
2009-04-10,255.3080
2009-04-11,626.0590
2009-04-12,209.2520
2009-04-13,152.8340
